# Assignment 1

## Part 1: K-nearest neighbours
Implement the k-nearest neighbors (k-NN) algorithm from scratch.  This involves writing the core k-NN functionality, including calculating the distances between data points and making predictions based on a majority vote among the k-nearest neighbors.

In [ ]:
class KNN:
    def __init__(self,k=3,dist_metric = "euclidean"):
        self.k = k
        pass